<a href="https://colab.research.google.com/github/hyydrra/PCA-and-LDA-on-FMNIST/blob/master/IS_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn import preprocessing
from scipy.stats import multivariate_normal
from numpy import linalg as LA
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

mnist_dataframe = pd.read_csv(
  "https://download.mlcc.google.com/mledu-datasets/mnist_train_small.csv",
  sep=",",
  header=None)

# Use just the first 10,000 records for training/validation.
mnist_dataframe = mnist_dataframe.head(10000)
mnist_dataframe = mnist_dataframe.reindex(np.random.permutation(mnist_dataframe.index))


In [ ]:
def parse_labels_and_features(dataset):
  """Extracts labels and features.
  
  This is a good place to scale or transform the features if needed.
  
  Args:
    dataset: A Pandas `Dataframe`, containing the label on the first column and
      monochrome pixel values on the remaining columns, in row major order.
  Returns:
    A `tuple` `(labels, features)`:
      labels: A Pandas `Series`.
      features: A Pandas `DataFrame`.
  """
  labels = dataset[0]

  # DataFrame.loc index ranges are inclusive at both ends.
  features = dataset.loc[:,1:784]
  # Scale the data to [0, 1] by dividing out the max value, 255.
  features = features / 255
  #x = StandardScaler().fit_transform(features)
  #feat_cols = ['feature'+str(i) for i in range(x.shape[1])]
  #normalised_breast = pd.DataFrame(x,columns=feat_cols)
  return labels, features

In [ ]:
training_labels, norm_train = parse_labels_and_features(mnist_dataframe[:7500])
test_labels, norm_test= parse_labels_and_features(mnist_dataframe[7500:10000])
test_labelss, norm_testt= parse_labels_and_features(mnist_dataframe[7500:10000])

norm_train.reset_index(inplace = True) 
norm_train = norm_train.drop("index",axis=1)
norm_test.reset_index(inplace = True) 
norm_test = norm_test.drop("index",axis=1)

In [ ]:
train_labels=training_labels.values.tolist()
test_labels=test_labels.values.tolist()
dim=norm_test.shape[1]
size_traindata=norm_train.shape[0]
size_testdata=norm_test.shape[0]

In [ ]:
train_classes={}
for i in range(10):
  train_classes[i]=[]
for i in range(len(train_labels)):
  train_classes[int(train_labels[i])].append(i)

len_tran_claases={}
for i in range(10):
  len_tran_claases[i]=len(train_classes[i])


In [ ]:
def mean_var(class_data):
  means = class_data.mean(axis=0)

  subtract = class_data-means
  #print(means)
  variance = np.zeros((class_data.shape[1],class_data.shape[1]))
  for i in range(class_data.shape[0]):
    x = [list(subtract.iloc[i])]
    xT = list(map(list, zip(*x)))
    temp= np.dot(xT,x)
    variance = variance + temp
    #print(variance)
  return means,(variance/(class_data.shape[0]))

def g_dist(train_data):
  dim=train_data.shape[1]
  norm_train = train_data
  G_dist={}
  for j in range(10):
    a=0.8
    mean,va=mean_var(norm_train.loc[train_classes[j], :] )
    va = ((1-a)*va) + (a*(va.trace())/dim)*np.eye(dim, dtype=float)
    var = multivariate_normal(mean=mean, cov=va)
    G_dist[j] = var
  return G_dist

In [ ]:
def validation(G_dist,norm_test):
  correct=0
  conf_matrix=[]
  for i in range(10):
    conf_matrix.append([0,0,0,0,0,0,0,0,0,0])

  for i in range(len(test_labels)):
    max=-100
    maxclass=None
    inp = list(norm_test.iloc[i])
    for j in range(10):
      pdf = G_dist[j].pdf(inp)*len(train_classes[j])
      if pdf>=max:
        max = pdf
        maxclass = j
    if maxclass==test_labels[i]:
      conf_matrix[int(test_labels[i])][int(test_labels[i])] = conf_matrix[int(test_labels[i])][int(test_labels[i])] +1
      correct = correct +1
    else:
      conf_matrix[int(test_labels[i])][maxclass] = conf_matrix[int(test_labels[i])][maxclass] +1
  return conf_matrix,correct


In [ ]:
G_dist =g_dist(norm_train)
conf_matrix,correct = validation(G_dist,norm_test)
print("percision without prior knowledge: ",correct/norm_test.shape[0])
conf_matrix

percision without prior knowledge:  0.9268


[[216, 1, 0, 0, 0, 0, 4, 0, 3, 1],
 [0, 287, 0, 0, 0, 0, 0, 0, 1, 0],
 [2, 7, 209, 3, 3, 0, 1, 7, 6, 4],
 [0, 6, 3, 228, 0, 1, 0, 3, 2, 3],
 [0, 7, 0, 0, 222, 0, 0, 2, 1, 6],
 [0, 3, 0, 13, 3, 181, 1, 0, 3, 4],
 [0, 3, 0, 0, 0, 2, 263, 0, 1, 0],
 [0, 6, 0, 1, 2, 0, 0, 250, 3, 5],
 [1, 16, 1, 8, 1, 0, 0, 1, 225, 8],
 [1, 4, 0, 3, 2, 0, 0, 7, 3, 236]]

# **PCA**

In [ ]:
mean,va=mean_var(norm_train)

In [ ]:
val, vec = LA.eig(va)
kvec_T = vec[:,0:40].T

In [ ]:
PCA_train_data = []
PCA_test_data = []
for j in range(len(train_labels)):
    PCA_train_data.append(np.matmul(kvec_T, norm_train.iloc[j].T))

for j in range(len(test_labels)):
    PCA_test_data.append(np.matmul(kvec_T, norm_test.iloc[j].T))

PCA_train_data = np.asanyarray(PCA_train_data).real 
PCA_test_data = np.asanyarray(PCA_test_data).real


In [ ]:
feat_cols = ['feature'+str(i) for i in range(PCA_train_data.shape[1])]
PCA_train_data = pd.DataFrame(PCA_train_data,columns=feat_cols)

feat_cols = ['feature'+str(i) for i in range(PCA_test_data.shape[1])]
PCA_test_data = pd.DataFrame(PCA_test_data,columns=feat_cols)


In [ ]:
PCA_norm_train=((PCA_train_data-PCA_train_data.min())/(1*(PCA_train_data.max()-PCA_train_data.min())))
PCA_norm_test=((PCA_test_data-PCA_test_data.min())/(1*(PCA_test_data.max()-PCA_test_data.min())))


In [ ]:
G_dist =g_dist(PCA_norm_train)
conf_matrix,correct = validation(G_dist,PCA_norm_test)
print("Accuracy without prior knowledge using PCA: ",correct/PCA_norm_test.shape[0])
conf_matrix

Accuracy without prior knowledge using PCA:  0.9164


[[221, 0, 0, 0, 0, 1, 2, 0, 1, 0],
 [0, 277, 3, 2, 0, 4, 0, 0, 2, 0],
 [4, 2, 219, 1, 1, 0, 1, 6, 6, 2],
 [1, 1, 9, 223, 0, 5, 1, 2, 3, 1],
 [1, 1, 1, 0, 212, 0, 1, 1, 6, 15],
 [4, 0, 0, 11, 1, 189, 1, 0, 1, 1],
 [6, 0, 1, 0, 0, 5, 256, 0, 1, 0],
 [1, 1, 3, 2, 4, 0, 0, 246, 2, 8],
 [2, 5, 1, 13, 1, 5, 0, 2, 227, 5],
 [4, 3, 0, 11, 4, 0, 0, 5, 8, 221]]

**LDA**

In [ ]:
mean_train = []
#norm_train = np.asanyarray(norm_train)
classes = {0:[], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[], 8:[], 9:[]}
scatterI = {0:[], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[], 8:[], 9:[]}

for i in range (len(train_labels)):
    classes[train_labels[i]].append(norm_train.iloc[i])
    
for i in range(10):
    mean_train.append(np.mean(classes[i], axis = 0))
    
    
for i in range(10):
    scatter = np.zeros([norm_train.shape[1], norm_train.shape[1]])
    for j in range(len(classes[i])):
        tmp = np.array([np.subtract(classes[i][j], mean_train[i])])
        tmp2 = tmp.T
        scatter += np.multiply(tmp, tmp2)
    scatterI[i] = (scatter)

scatterW = np.zeros([norm_train.shape[1], norm_train.shape[1]])
for i in range(10):
    scatterW = np.add(scatterW, scatterI[i])

meanAll = np.mean(norm_train, axis = 0)

scatterB = np.zeros([norm_train.shape[1], norm_train.shape[1]])

for i in range(10):
    tmp = np.array([np.subtract(mean_train[i], meanAll)])
    tmp2 = tmp.T
    scatterB += np.multiply(tmp, tmp2)


a=0.8 
temp =  ((1-a)*scatterW) + (a*(scatterW.trace())/dim)*np.eye(dim, dtype=float)  
invscatterW = np.linalg.inv(temp)

ScatterP = np.matmul(invscatterW, scatterB)



In [ ]:
val1, vec1 = LA.eig(ScatterP)
kvec_T1 = vec1[:,0:40].T

In [ ]:
LDA_train_data = []
LDA_test_data = []
for j in range(len(train_labels)):
    LDA_train_data.append(np.matmul(kvec_T1, norm_train.iloc[j].T))

for j in range(len(test_labels)):
    LDA_test_data.append(np.matmul(kvec_T1, norm_test.iloc[j].T))

LDA_train_data = np.asanyarray(LDA_train_data).real 
LDA_test_data = np.asanyarray(LDA_test_data).real

feat_cols = ['feature'+str(i) for i in range(LDA_train_data.shape[1])]
LDA_train_data = pd.DataFrame(LDA_train_data,columns=feat_cols)
feat_cols = ['feature'+str(i) for i in range(LDA_test_data.shape[1])]
LDA_test_data = pd.DataFrame(LDA_test_data,columns=feat_cols)

LDA_norm_train=((LDA_train_data-LDA_train_data.min())/(1*(LDA_train_data.max()-LDA_train_data.min())))
LDA_norm_test=((LDA_test_data-LDA_test_data.min())/(1*(LDA_test_data.max()-LDA_test_data.min())))


G_dist1 = g_dist(LDA_norm_train)

LDA_Correct = 0
conf_matrix=[]
for i in range(10):
  conf_matrix.append([0,0,0,0,0,0,0,0,0,0])

for i in range(len(test_labels)):
  maxx=0
  maxclass=None
  inp = list(LDA_norm_test.iloc[i])
  for j in range(10):
    pdf = G_dist1[j].pdf(inp)*len(train_classes[j])
    if pdf>=maxx:
      maxx = pdf
      maxclass = j
  if maxclass==test_labelss.iloc[i]:
    conf_matrix[int(test_labelss.iloc[i])][int(test_labelss.iloc[i])] = conf_matrix[int(test_labelss.iloc[i])][int(test_labelss.iloc[i])] +1
    LDA_Correct = LDA_Correct +1
  else:
    conf_matrix[int(test_labelss.iloc[i])][maxclass] = conf_matrix[int(test_labelss.iloc[i])][maxclass] +1

print("Accuracy without prior knowledge using LDA: ",LDA_Correct/test_labelss.shape[0])
conf_matrix

Accuracy without prior knowledge using LDA:  0.8896


[[213, 0, 0, 0, 1, 2, 7, 0, 2, 0],
 [0, 278, 0, 2, 0, 0, 1, 0, 7, 0],
 [4, 9, 194, 5, 2, 1, 7, 5, 13, 2],
 [0, 2, 6, 216, 0, 8, 3, 2, 6, 3],
 [0, 2, 0, 1, 211, 0, 2, 1, 5, 16],
 [6, 2, 0, 10, 4, 174, 0, 1, 5, 6],
 [2, 2, 0, 0, 2, 5, 256, 0, 2, 0],
 [1, 7, 4, 1, 5, 0, 1, 238, 2, 8],
 [2, 5, 0, 10, 1, 9, 1, 0, 225, 8],
 [4, 2, 1, 9, 9, 1, 0, 2, 9, 219]]